<a href="https://colab.research.google.com/github/iwasakishuto/TeiLab-BasicLaboratoryWork-in-LifeScienceExperiments/blob/develop/notebook/Colaboratory/microarray2020A_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## マイクロアレイデータ解析

- 実習のwiki: [http://bit.ly/microarray-2020](http://bit.ly/microarray-2020)
- 元論文: [Functional dissection of siRNA sequence...](2020-10-26-15-45.html)

マイクロアレイを用いた網羅的遺伝子発現解析

網羅的に遺伝子の発現を解析する手法であるマイクロアレイの原理、操作手順、解析方法について学ぶ。

siRNAを導入した細胞からRNAを抽出し、マイクロアレイを行うことで、全mRNAの変動量を解析する。

#### はじめに

今回Pythonの実習を担当させていただく、程研B4の岩崎（生情）です。

この実習が終わったときに、**「マイクロアレイ解析結構Pythonでできるし、あんなことやこんなことも自分でプログラムを書いて調べて見たい！！」** と思ってもらえることを目指して頑張りますので、わからないところがあれば、遠慮なく中断してください！！

また、zoomのチャットや、[TwitterのDM](https://www.twitter.com/messages/compose?recipient_id=1042783905697288193&text=%E3%80%87%E3%80%87%E3%81%8C%E3%82%8F%E3%81%8B%E3%82%89%E3%81%AA%E3%81%84%E3%81%AE%E3%81%A7%E3%80%81%E3%82%82%E3%81%86%E4%B8%80%E5%BA%A6%E8%AA%AC%E6%98%8E%E3%82%92%E3%81%8A%E9%A1%98%E3%81%84%E3%81%97%E3%81%BE%E3%81%99%E3%80%82%E7%AD%89)等も適宜確認しながら進めるつもりなので、何かあればご連絡ください🙇‍♂️

### 0. 環境構築

まず、[程研wiki](http://ui-tei.rnai.jp/microarray/doku.php?do=show&id=microarray2020_teilab)の**アレイデータファイル**から[実習解析用データ.zip](https://drive.google.com/file/d/1xUCG7WZWHFmWFRy0OjgJr_-GKCSJrv2C/view)をダウンロードする。

ここで、データが大きいため、[gdown.pl](https://github.com/circulosmeos/gdown.pl)というツールを用いることとする。（Googleローカルで動かしている場合は、`$ pip install gdown` でインストール可能）

In [ ]:
! ls

__MACOSX  sample_data  seedmatch.txt  実習解析用データ	実習解析用データ.zip


In [ ]:
! gdown --id "18fMyp6liYh6dT0DxitYZ_h9CMJZCxSo6" -O "実習解析用データ.zip"
! unzip "実習解析用データ.zip"

In [ ]:
! ls

__MACOSX  sample_data  seedmatch.txt  実習解析用データ	実習解析用データ.zip


次に、上記のデータを読み込み、全サンプルのデータを1つのテーブルにまとめる。

In [ ]:
dirname = "実習解析用データ"

In [ ]:
! apt-get install tree

In [ ]:
! tree $dirname

実習解析用データ
├── US91503671_253949442637_S01_GE1_105_Dec08_1_1.pdf
├── US91503671_253949442637_S01_GE1_105_Dec08_1_1.txt
├── US91503671_253949442637_S01_GE1_105_Dec08_1_2.pdf
├── US91503671_253949442637_S01_GE1_105_Dec08_1_2.txt
├── US91503671_253949442637_S01_GE1_105_Dec08_1_3.pdf
├── US91503671_253949442637_S01_GE1_105_Dec08_1_3.txt
├── US91503671_253949442637_S01_GE1_105_Dec08_1_4.pdf
├── US91503671_253949442637_S01_GE1_105_Dec08_1_4.txt
├── US91503671_253949442637_S01_GE1_105_Dec08_2_1.pdf
├── US91503671_253949442637_S01_GE1_105_Dec08_2_1.txt
├── US91503671_253949442637_S01_GE1_105_Dec08_2_2.pdf
├── US91503671_253949442637_S01_GE1_105_Dec08_2_2.txt
├── US91503671_253949442637_S01_GE1_105_Dec08_2_3.pdf
├── US91503671_253949442637_S01_GE1_105_Dec08_2_3.txt
├── US91503671_253949442637_S01_GE1_105_Dec08_2_4.pdf
└── US91503671_253949442637_S01_GE1_105_Dec08_2_4.txt

0 directories, 16 files






|ファイル名	|サンプル	|班|
|:-:|:-:|:-:|
|US91503671_253949442637_S01_GE1_105_Dec08_1_1.txt|	mock	|1班|
|US91503671_253949442637_S01_GE1_105_Dec08_1_2.txt|	RNA	  |2班|
|US91503671_253949442637_S01_GE1_105_Dec08_1_3.txt|	2OMe3	|3班|
|US91503671_253949442637_S01_GE1_105_Dec08_1_4.txt|	2OMe5	|4班|
|US91503671_253949442637_S01_GE1_105_Dec08_2_1.txt|	mock	|5班|
|US91503671_253949442637_S01_GE1_105_Dec08_2_2.txt|	2OMe7	|6班|
|US91503671_253949442637_S01_GE1_105_Dec08_2_3.txt|	LNA3	|7班|
|US91503671_253949442637_S01_GE1_105_Dec08_2_4.txt|	LNA7	|8班|

それでは、実際にデータをpythonで読み込んでいく。

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

In [ ]:
print(sys.version)

3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0]


In [ ]:
file_paths = sorted([os.path.join(dirname,path) for path in os.listdir(dirname) if os.path.splitext(path)[1] == ".txt"])
sample_names = ["mock(1)","RNA","2OMe3","2OMe5","mock(5)","2OMe7","LNA3","LNA7"]

### 1. データの読み込み

全サンプルのデータを1つのテーブルにまとめる。

なお、**行番号とプローブ番号の対応関係はどのサンプルも同じである** ため、サンプル $X$ のデータの隣にサンプル $Y$ のデータを `concatenate` すれば1枚のワークシートにまとまる。

これを繰り返して、全サンプルの`gProcessedSignal`, `gIsWellAboveBG` の値を1 つのテーブルにまとめる。

In [ ]:
! head -n20 {file_paths[0]}

TYPE	text	text	text	text	integer	float	float	text	text	text	integer	integer	integer	integer	float	float	float	float	float	float	text	text	text	text	text	text	text	text	text	text	text	text	integer	integer	text	integer	integer	integer	float	float	float	float	float	float	float	float	integer	integer	float	integer	float	float	float	float	integer	float	integer	float	integer	text	integer	integer	float	float	integer	float	float	float	float	float	float	float	integer	integer	integer	float	float	integer	float	float	integer	float	integer	integer	float	integer	integer	text	integer	float	float	integer	float	float	float	float	integer	integer	float	float	integer	integer	integer	integer	integer	float	integer	integer	float	integer	integer	integer	integer	float	float	integer	integer	integer	integer	integer	integer	integer	float	float	integer	float	integer	integer	float	integer	float	integer	float	float	float	integer	text	integer	integer	float	float	float	float	float	float	integer	integer	float	text	integ

中身を見たところ、0-7行目がメタデータ、8行目からが各プローブのデータになっていることがわかる。したがって、`pd.read_csv(filepath, sep="\t", header=9)` (or `pd.read_table(filepath, header=9)`) とすれば良いことがわかる。

まず、各プローブのアノテーションデータ（どういうプローブかの説明。以下のカラム）を取り出す。なお、このデータはサンプルに寄らないので、8つのサンプルのうち一つから一度だけ取り出す。

In [ ]:
USE_COLS_ANNO  = [
    "FeatureNum", "ControlType", "ProbeName", "GeneName", "SystematicName"
]

|column name|description|
|:-:|:-|
|`FeatureNum`|スポットの番号|
|`ControlType`|<ul><li>positive controlは `1`</li> <li>negative controlは `-1`</li><li>それ以外（解析で用いる）は `0`</li></ul>|
|`ProbeName`|プローブ名|
|`GeneName`|遺伝子名|
|`SystematicName`|遺伝子名|

In [ ]:
df_anno = pd.read_csv(file_paths[0], sep="\t", header=9, usecols=USE_COLS_ANNO)
df_anno.head(5)

,FeatureNum,ControlType,ProbeName,GeneName,SystematicName
0,1,1,GE_BrightCorner,GE_BrightCorner,GE_BrightCorner
1,2,1,DarkCorner,DarkCorner,DarkCorner
2,3,1,DarkCorner,DarkCorner,DarkCorner
3,4,0,A_23_P117082,HEBP1,NM_015987
4,5,0,A_33_P3246448,KCNE4,NM_080671



続いて、各サンプルのシグナル強度(`gProcessedSignal`)のデータを取得する。なお、この時 `gIsWellAboveBG` が `0` のものは**（真の）シグナルがバックグラウンドのシグナルよりも低く、信頼できないデータであるため、取り除く。**

In [ ]:
USE_COLS_SYGNAL = [
    "gProcessedSignal", "gIsWellAboveBG"
]

|column name|description|
|:-:|:-|
|`gProcessedSignal`|green(Cy-3)のシグナル強度（＝発現量）|
|`gIsWellAboveBG`|（真の）シグナルがバックグラウンドのシグナルより十分高いか？（＝信頼できるデータか）|

In [ ]:
df_combined = df_anno.copy(deep=True)
index = set(df_combined.index)
print(f"データ数(before): {len(df_combined)}")

for fp,name in zip(file_paths, sample_names):
  df_signal = pd.read_csv(fp, sep="\t", header=9, usecols=USE_COLS_SYGNAL)
  index = index & set(df_signal[(df_signal.gIsWellAboveBG==1)].index)
  df_combined = pd.concat([df_combined, df_signal[["gProcessedSignal"]].rename(columns={"gProcessedSignal" : name})], axis=1)

df_filtered_1 = df_combined.loc[index, :]
print(f"データ数(after) : {len(df_filtered_1)}")

データ数(before): 62976
データ数(after) : 21798


また、`ControlType` の値が $\pm1$ のものはコントロールであるため、`0` のもののみ取り出す。

In [ ]:
print("データ数(before):", len(df_filtered_1))
df_filtered_2 = df_filtered_1[df_filtered_1.ControlType == 0]
print("データ数(after) :", len(df_filtered_2))

データ数(before): 21798
データ数(after) : 21511


In [ ]:
# インデックスを振り直す。
df_filtered = df_filtered_2.reset_index(drop=True)
df_filtered.head(5)

,FeatureNum,ControlType,ProbeName,GeneName,SystematicName,mock(1),RNA,2OMe3,2OMe5,mock(5),2OMe7,LNA3,LNA7
0,32769,0,A_23_P29036,IFNGR2,NM_005534,410.3776,400.4074,411.7127,436.8438,394.3873,468.7426,377.9030,336.3862
1,32771,0,A_23_P10385,DTL,NM_016448,2125.9110,1937.1130,1910.0520,1920.9070,1874.9810,1907.1360,2062.7130,2003.6830
2,4,0,A_23_P117082,HEBP1,NM_015987,12566.8700,12356.9600,12896.7400,12860.2400,12416.2700,13869.8100,14213.9600,13553.8400
3,32772,0,A_21_P0009820,XLOC_013737,ENST00000422519,249.2575,210.7532,233.8117,253.5205,261.4639,231.3730,284.0021,303.7988
4,32773,0,A_23_P1072,ATP1A1,NM_000701,16938.0000,16912.6900,16270.8600,16853.2400,16152.8900,15276.8400,16924.8800,13075.0700


```python
# データをGoogleDriveに保存したい場合は、以下のコードを走らせてください。
from google.colab import drive
drive.mount('./gdrive')
df_filtered.to_excel("./gdrive/My Drive/microarray_filtered.xlsx", index=False)
```

In [ ]:
# mockは、単純に平均をとる。
df_filtered["mock"] = df_filtered[["mock(1)", "mock(5)"]].mean(1)

### 2. 解析

ここでは、XYプロットとMAプロットを図示し、シグナル強度の分布を調べる。

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly import offline
from plotly.subplots import make_subplots

#### 2.1 X-Y プロット

- サンプル $X$ の `gProcessedSignal` の値を横軸
- サンプル $Y$ の `gProcessedSignal` の値を縦軸

にプロットした以下のものを **X-Yプロット** と呼ぶ。

In [ ]:
def XYplot(df, x, y, hover_data=[]):
    fig = px.scatter(
        df, x=x, y=y, hover_data=hover_data,
        title=f"XY plot ({x} vs {y})"
    )
    return fig

In [ ]:
fig = XYplot(df=df_filtered, x="mock", y="RNA", hover_data=["GeneName"])
fig.show()

> VIM はどこにあると予想できますか？？実際に確認してみてください！

#### 2.2 MAプロット

- $log_{10}(XY)$ を横軸
- $log_2(Y/X)$ を縦軸

にプロットしたものである。

In [ ]:
def MAplot(df, control, target, hover_data=[]):
  control_signals = df[control].values
  target_signals  = df[target].values
  X = np.log10(control_signals*target_signals)
  Y = np.log2(target_signals/control_signals)
  
  fig = go.Figure(data=go.Scatter(
      x=X, y=Y, hovertext=df[hover_data].values, mode="markers", marker_size=3
  ))
  fig.update_layout(
      title=f"MA plot ({control} vs {target})",
      xaxis_title = "$log_{10}" + f"({target} * {control})$", 
      yaxis_title = f"$log_2{target}/{control}$",
      
  )
  return fig

In [ ]:
fig = MAplot(df=df_filtered, control="mock", target="RNA", hover_data="GeneName")
fig.show()

> - VIM はどこにあると予想できますか？？実際に確認してみてください！
> - 縦軸が $0$ / $1$ / $-1$  とは何を意味しますか？？
> - 横軸は何を意味しますか？？

### 3. 発展

#### 3.1 データの正規化

In [ ]:
# 累積度数をプロットする。
def plotDensities(data, names=sample_names, col=1, fig=None, title=""):
    fig = fig or make_subplots(rows=1, cols=1)
    for ith_data,name in zip(np.log2(data),names):
        hist, bin_edges = np.histogram(a=ith_data, bins=100, density=True)
        fig.add_trace(trace=go.Scatter(x=bin_edges[1:], y=hist, name=name, mode="lines"), row=1, col=col)
    fig.update_layout(
        title=title,
        xaxis_title = "$log_2(gProcessedSignal)$", 
        yaxis_title = "Density",
        
    )
    return fig

In [ ]:
raw_data = df_filtered[sample_names].T.values
print(f"raw_data.shape = {raw_data.shape}")

raw_data.shape = (8, 21511)


In [ ]:
fig = plotDensities(raw_data, names=sample_names, title="raw data.")
fig.show()

#### 3.1.1 75%tile

各サンプルごとに、発現量の値を順番に並べ替え、発現量の**小さい方から数えて**順位75%に位置するものの値を求める。

この75%tileの値は通常サンプルごとに異なるが、それらの相乗平均 $a$ を求める。

各サンプルごとに、全プローブの値に 「$a$/そのサンプルにおける75%tileの値」をかける（つまり、全サンプルで75%tile値を $a$ に揃える。）

In [ ]:
from scipy.stats.mstats import gmean

def tile75_normalization(data):
    a = np.percentile(a=data, q=75, axis=1)
    return data*(a/gmean(a))[:,np.newaxis]

In [ ]:
tile75_data = tile75_normalization(raw_data)
print(f"tile75_data.shape = {tile75_data.shape}")

tile75_data.shape = (8, 21511)


In [ ]:
fig = plotDensities(tile75_data, names=sample_names, title="75%tile")
fig.show()

#### 3.1.2 quantile法

各サンプルごとに、発現量の値を順番に並べ替え、各順位の値をそれぞれ同順位のシグナル値の相乗平均で置き換える。

その結果、全サンプルで分布が同一になる。

In [ ]:
def quantile_normalization(data):
    return gmean(a=np.sort(a=data, axis=1), axis=0)[np.argsort(np.argsort(data, axis=1), axis=1)]

In [ ]:
quantiled_data = quantile_normalization(raw_data)
print(f"quantiled_data.shape = {quantiled_data.shape}")

quantiled_data.shape = (8, 21511)


In [ ]:
fig = plotDensities(quantiled_data, names=sample_names, title="quantile")
fig.show()

#### 3.2.1 正規化法の比較

各正規化処理後のデータを比較する。

In [ ]:
fig = make_subplots(rows=1, cols=3)
plotDensities(raw_data,       names=sample_names, fig=fig, col=1)
plotDensities(tile75_data,    names=sample_names, fig=fig, col=2)
plotDensities(quantiled_data, names=sample_names, title="Comparison of normalization methods", fig=fig, col=3)
fig.show()

#### 3.2 seedマッチする遺伝子群の累積度数を調べる

入力した配列を3'UTRにもつ遺伝子（アクセッション番号）のリストを表示するページ（[seedmatch](http://atlas.RNAi.jp/seedmatch/)）を用いて、siRNAのガイド鎖のseed（`UGAACUC`）と相補的な配列（`GAGTTCA`）が3'UTRに存在する遺伝子を検索する。

In [ ]:
! gdown --id "0B1jjJ8m0VChCY28xRmhVcTRoamc" -O "seedmatch.txt"

Downloading...
From: https://drive.google.com/uc?id=0B1jjJ8m0VChCY28xRmhVcTRoamc
To: /content/seedmatch.txt
100% 48.8k/48.8k [00:00<00:00, 18.4MB/s]


In [ ]:
df_matched_mRNAs = pd.read_csv("seedmatch.txt", sep="\t")

In [ ]:
df_matched_mRNAs.columns = ["SystematicName", "NumHits"]

In [ ]:
# seedmatchで検索したデータと紐付ける。
df_is_matched = pd.merge(df_filtered, df_matched_mRNAs, on="SystematicName", how="left").fillna(0)

In [ ]:
# 累積度数曲線を描くために、ソートする。
df_is_matched["log2(RNA/mock)"] = np.log2(df_is_matched["RNA"]/df_is_matched["mock"])
df_is_matched = df_is_matched.sort_values(by="log2(RNA/mock)").reset_index(drop=False)
df_is_matched.head(5)

,index,FeatureNum,ControlType,ProbeName,GeneName,SystematicName,mock(1),RNA,2OMe3,2OMe5,mock(5),2OMe7,LNA3,LNA7,mock,NumHits,log2(RNA/mock)
0,5045,40255,0,A_23_P161190,VIM,NM_003380,113061.4000,11273.03000,13240.8100,13071.9200,97095.4100,19334.3000,28738.2500,107303.0000,105078.40500,0.0,-3.220519
1,2559,3725,0,A_24_P396231,LAMP2,NM_002294,450.9737,95.27578,277.0111,283.0721,445.2353,390.5766,338.8382,415.8079,448.10450,2.0,-2.233654
2,19353,28371,0,A_24_P4705,PPME1,NM_016147,822.7149,246.87330,376.0636,331.0058,830.8831,377.5120,330.5200,354.1883,826.79900,1.0,-1.743766
3,12105,17634,0,A_23_P146417,C9orf5,NM_032012,3189.5740,976.94710,2191.7760,1788.6940,3145.1030,3437.1110,3127.5030,3795.3370,3167.33850,2.0,-1.696919
4,7088,10343,0,A_33_P3413701,ERAP1,NM_001040458,609.4255,189.86100,530.7483,431.6865,592.7684,595.4490,478.5547,563.1971,601.09695,1.0,-1.662654


In [ ]:
def CFC_trace_create(data, name=""):
    """Create a CFC(Cumulative Frequency Curve)"""
    num_data = len(data)
    trace = go.Scatter(x=data, y=[(i+1)/num_data for i in range(num_data)], mode="lines", name=name)
    return trace

In [ ]:
fig = {
    "data":[
        CFC_trace_create(data=df_is_matched["log2(RNA/mock)"].values, name="all"),
        CFC_trace_create(data=df_is_matched[df_is_matched["NumHits"]!=0]["log2(RNA/mock)"].values, name="seed matched mRNAs")
    ],
    "layout": go.Layout(title="The expression level(All vs seed matched mRNAs)", xaxis_title="$log_2(RNA/mock)$", yaxis_title="Cumulative frequency")
}
offline.iplot(fig)

#### 3.3 seedマッチで色々遊んでみる。

- マッチするシードの数（`NumHits`）って、多い方が抑制されてる？？
- そもそもなんで2-8の7merでオフターゲット効果が起きるの？？
  - 1-7や3-9は？？
  - 6merや9merは？？
  - Argonauteタンパク質と結合（loading）し、RISC(RNA induced silencing complex)を形成するが、構造的に…
- 統計的に有意だと言える？？

In [ ]:
import re
import requests

In [ ]:
df_filtered["log2(RNA/mock)"] = np.log2(df_filtered["RNA"]/df_filtered["mock"])

In [ ]:
def get_matched_mRNAs(seedseq):
    url = "http://atlas.rnai.jp/seedmatch/seedmatch.cgi"
    ret = requests.post(url=url, data={"seedseq": seedseq.lower()})
    df_match = pd.DataFrame(
        data=re.findall(pattern=r"\n(.+)\t([0-9]+)", string=ret.text),
        columns=["SystematicName", "NumHits"]
    )
    return df_match

In [ ]:
df_matched_mRNAs = get_matched_mRNAs(seedseq="GAGTTCA")
df_matched_mRNAs.head(5)

,SystematicName,NumHits
0,NM_001004713,1
1,NM_173860,1
2,NM_001005493,1
3,NM_001005516,1
4,NM_007084,1
